In [1]:
# Import the necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output

class ClippedIntText(widgets.IntText):
    def __init__(self, *args, **kwargs):
        self._min = kwargs.pop('min', None)
        self._max = kwargs.pop('max', None)
        super().__init__(*args, **kwargs)
        self.observe(self.validate, 'value')

    def validate(self, change):
        if self._min is not None and change['new'] < self._min:
            self.value = self._min
        elif self._max is not None and change['new'] > self._max:
            self.value = self._max


class ClippedFloatText(widgets.FloatText):
    def __init__(self, *args, **kwargs):
        self._min = kwargs.pop('min', None)
        self._max = kwargs.pop('max', None)
        super().__init__(*args, **kwargs)
        self.observe(self.validate, 'value')

    def validate(self, change):
        if self._min is not None and change['new'] < self._min:
            self.value = self._min
        elif self._max is not None and change['new'] > self._max:
            self.value = self._max


# Create widgets for input of number of samples and components
num_samples_input = ClippedIntText(description='Number of Samples:', value=1, min=1, max=10)
num_components_input = ClippedIntText(description='Number of Components:', value=1, min=1, max=10)

# Create output widgets where the form will be displayed
output_components = widgets.VBox()
output_samples = widgets.VBox()

# Define the units that users can select
units = ['nM', 'uM', 'mM', 'M']

# Initialize the list of component widgets and sample widgets
component_widgets = [{'name': widgets.Text(description='Component 1'),
                      'initial_conc': widgets.FloatText(description='Initial Conc.'),
                      'unit': widgets.Dropdown(options=units, description='Unit:')
                     }]
sample_widgets = [{'name': widgets.Text(description='Sample 1'),
                   'target_concs': [widgets.FloatText(description='Component 1')]
                  }]

# Code for updating unit dropdowns when a component's unit changes
def on_unit_change(change):
    component_index = change['owner'].component_index
    new_unit_index = units.index(change['new'])
    for sample in sample_widgets:
        sample['units'][component_index].options = units[:new_unit_index+1]

# Update the list of component widgets when the number of components changes
def on_num_components_change(change):
    num_components = change['new']
    # Prevent components being < 1 or > 10
    if num_components < 1:
        num_components_input.set_value(1)
        return
    elif num_components > 10:
        num_components_input.set_value(10)
        return
    while len(component_widgets) < num_components:
        i = len(component_widgets) + 1
        unit_dropdown = widgets.Dropdown(options=units, description='Unit:', component_index=i-1)
        unit_dropdown.observe(on_unit_change, names='value')
        component_widgets.append({
            'name': widgets.Text(description=f'Component {i}'),
            'initial_conc': widgets.FloatText(description='Initial Conc.'),
            'unit': unit_dropdown
        })
    while len(component_widgets) > num_components:
        component_widgets.pop()

    # Update the target concentration inputs and units in the sample form
    for sample in sample_widgets:
        while len(sample['target_concs']) < num_components:
            sample['target_concs'].append(widgets.FloatText(description=f'Component {len(sample["target_concs"]) + 1}'))
            sample['units'].append(widgets.Dropdown(options=units[:component_widgets[len(sample["target_concs"])]['unit'].index+1], description=''))
        while len(sample['target_concs']) > num_components:
            sample['target_concs'].pop()
            sample['units'].pop()

    # Refresh the display
    display_forms()

# Update the list of sample widgets when the number of samples changes
def on_num_samples_change(change):
    num_samples = change['new']
    # Prevent samples being < 1 or > 10
    if num_samples < 1:
        num_samples_input.set_value(1)
        return
    elif num_samples > 10:
        num_samples_input.set_value(10)
        return
    while len(sample_widgets) < num_samples:
        i = len(sample_widgets) + 1
        sample_widgets.append({
            'name': widgets.Text(description=f'Sample {i}'),
            'target_concs': [widgets.FloatText(description=f'Component {j+1}') for j in range(len(component_widgets))],
            'units': [widgets.Dropdown(options=units[:component_widgets[j]['unit'].index+1], description='') for j in range(len(component_widgets))]
        })
    while len(sample_widgets) > num_samples:
        sample_widgets.pop()

    # Refresh the display
    display_forms()

# Create the input and sample forms based on the current component and sample widgets
def create_input_form():
    component_name_inputs = [component['name'] for component in component_widgets]
    initial_concentration_inputs = [widgets.HBox([component['initial_conc'], component['unit']]) for component in component_widgets]
    input_form = widgets.VBox([widgets.HBox([name_input, concentration_input]) for name_input, concentration_input in zip(component_name_inputs, initial_concentration_inputs)])
    return input_form

def create_sample_form():
    sample_name_inputs = [sample['name'] for sample in sample_widgets]
    component_concentration_inputs = [[target_conc for target_conc in sample['target_concs']] for sample in sample_widgets]
    sample_form = widgets.VBox([widgets.HBox([name_input, widgets.VBox(target_conc_inputs)]) for name_input, target_conc_inputs in zip(sample_name_inputs, component_concentration_inputs)])
    return sample_form

# Display the input and sample forms
def display_forms():
    # Create the input and sample forms
    input_form = create_input_form()
    sample_form = create_sample_form()

    # Update the children of the VBox widgets
    output_components.children = [input_form]
    output_samples.children = [sample_form]


# Listen for changes in the number of components and samples
num_components_input.observe(on_num_components_change, names='value')
num_samples_input.observe(on_num_samples_change, names='value')

# Display the widgets for entering the number of components and samples
display(num_components_input)
display(num_samples_input)

# Display the output widgets
display(output_components)
display(output_samples)

# Display the initial input and sample forms
display_forms()


ClippedIntText(value=1, description='Number of Components:')

ClippedIntText(value=1, description='Number of Samples:')

VBox()

VBox()

KeyError: 'units'